## 1. Install Required Libraries

In [ ]:
!pip install \
langchain_community \
unstructured \
langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
!pip install faiss-cpu sentence-transformers pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.7 MB/s eta 0:00:00


## 2. Import Required Libraries

In [ ]:
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import faiss
import numpy as np


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## 3: Load and Split Your PDF Document

In [ ]:
loader = PyPDFLoader("/content/Nepal.pdf")
docs = loader.load_and_split()

# Split documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter()
split_docs = text_splitter.split_documents(docs)


## 4: Generate Embeddings Using Sentence-Transformers

In [ ]:
# Load the pre-trained model for generating embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each split document
doc_embeddings = model.encode([doc.page_content for doc in split_docs])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 5: Set Up FAISS for Vector Search

In [13]:
# Initialize FAISS index for cosine similarity
embedding_dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatIP(embedding_dimension)  # Using Inner Product (cosine similarity) for better performance
faiss.normalize_L2(doc_embeddings)  # Normalize the embeddings to unit length for cosine similarity

# Add embeddings to the FAISS index
index.add(np.array(doc_embeddings))


## 6: Perform Similarity Search Using FAISS

In [27]:
# Define your query and generate its embedding
query = "waht is world highest mountain?"
query_embedding = model.encode([query])
faiss.normalize_L2(query_embedding)  # Normalize the query embedding

# Perform similarity search
k = 2  # Number of similar documents to retrieve
distances, indices = index.search(np.array(query_embedding), k)

# Retrieve the matched documents
similar_docs = [split_docs[idx] for idx in indices[0]]
# print(similar_docs[0].page_content)  # Print the content of the most similar document

# Function to dynamically generate a response based on the most relevant sentence
def generate_best_sentence_response(docs, query):
    combined_content = " ".join([doc.page_content for doc in docs])
    sentences = combined_content.split('. ')  # Split content into sentences

    # Generate embeddings for all sentences
    sentence_embeddings = model.encode(sentences)
    faiss.normalize_L2(sentence_embeddings)  # Normalize sentence embeddings for cosine similarity

    # Calculate cosine similarities between the query and each sentence
    similarities = np.dot(sentence_embeddings, query_embedding.T)

    # Find the most similar sentence
    most_relevant_idx = np.argmax(similarities)
    best_sentence = sentences[most_relevant_idx]

    if similarities[most_relevant_idx] < 0.5:  # Check against a similarity threshold
        response = "I'm sorry, I couldn't find any specific information related to your query in the provided documents."
    else:
        response = f"Here is the most relevant information regarding your query:\n\n{best_sentence.strip()}"

    return response

# Generate and print the response
chatbot_response = generate_best_sentence_response(similar_docs, query)
print(chatbot_response)

Here is the most relevant information regarding your query:

The allure of the Himalayas, including Mount Everest, the world's highest 
peak, and the Annapurna Circuit, one of the most challenging trekking routes, is a major draw for 
adventure seekers
